In [1]:
# !pip3 install sacrebleu

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [4]:
from transformers import T5Config, TFT5Model, T5Model, load_tf_weights_in_t5, T5Tokenizer
from transformers import T5ForConditionalGeneration, TFT5ForConditionalGeneration

In [5]:
model_tf = TFT5ForConditionalGeneration.from_pretrained('mesolitica/t5-tiny-finetuned-noisy-en-ms')

2022-08-01 01:34:19.890265: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-01 01:34:19.890297: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-08-01 01:34:19.890301: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-08-01 01:34:19.890419: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-08-01 01:34:19.890433: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-08-01 01:34:19.890436: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.129.6
2022-08-01 01:34:19.890631: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU i

In [6]:
tokenizer = T5Tokenizer.from_pretrained('mesolitica/t5-tiny-finetuned-noisy-en-ms')

In [7]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
right = [unidecode(s) for s in ms]
left = [unidecode(s) for s in eng]

In [8]:
len(ms), len(eng)

(998, 998)

In [9]:
from tqdm import tqdm

batch_size = 16

results = []
for i in tqdm(range(0, len(left), batch_size)):
    input_ids = [{'input_ids': tokenizer.encode(f'terjemah Inggeris ke Melayu: {s}', return_tensors = 'tf')[0]} for s in left[i:i + batch_size]]
    padded = tokenizer.pad(input_ids, padding = 'longest')
    outputs = model_tf.compat.v1.generate(**padded, max_length = 1000)
    for o in outputs:
        results.append(tokenizer.decode(o, skip_special_tokens=True))

  0%|                                                                                                         | 0/63 [00:00<?, ?it/s]2022-08-01 01:34:29.011302: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x9686860 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-01 01:34:29.011335: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): Host, Default Version
2022-08-01 01:34:29.068280: I tensorflow/compiler/jit/xla_compilation_cache.cc:363] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [02:21<00:00,  2.24s/it]


In [10]:
len(results)

998

In [11]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(right[no])

In [12]:
refs = [filtered_right]
sys = filtered_left

In [13]:
r = bleu.corpus_score(sys, refs)

In [14]:
r.__dict__

{'name': 'BLEU',
 'score': 39.32913677475762,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '71.2/47.3/32.8/23.0 (BP = 0.984 ratio = 0.985 hyp_len = 21687 ref_len = 22027)',
 'bp': 0.9844446589024665,
 'counts': [15439, 9787, 6465, 4308],
 'totals': [21687, 20690, 19693, 18696],
 'sys_len': 21687,
 'ref_len': 22027,
 'precisions': [71.1901138931157,
  47.30304494925085,
  32.828923983141216,
  23.042362002567394],
 'prec_str': '71.2/47.3/32.8/23.0',
 'ratio': 0.9845643982385255}

In [15]:
chrf.corpus_score(sys, refs)

chrF2++ = 64.26